In [2]:
!pip install -q transformers datasets accelerate
print("✅ Installed libraries")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigfram

In [3]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

print("✅ Libraries imported")


2025-06-06 09:08:50.586036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749200931.034829      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749200931.160626      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Libraries imported


In [4]:
dataset = load_dataset("emotion")
print(dataset)


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # pad token fix

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

print("✅ GPT-2 Mini loaded")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ GPT-2 Mini loaded


In [6]:
def add_emotion_tag(example):
    return {"text": f"<{example['label']}> {example['text']}"}

dataset = dataset.map(add_emotion_tag)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)
print(tokenized_datasets)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
})


In [7]:
from transformers import TrainingArguments, TrainerCallback, DataCollatorForLanguageModeling

# Custom callback to force printing logs
class PrintCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Step {state.global_step}: {logs}")

# Data collator for causal language modeling (GPT2)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # causal LM, not masked LM
)

# Training arguments with logging every 10 steps
training_args = TrainingArguments(
    output_dir="./gpt2-emotion",
    do_eval=True,
    eval_steps=500,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    no_cuda=False,  # Set True if you're not using a GPU
    report_to="none",  # Disable wandb logging
)

print("✅ Training setup ready")

✅ Training setup ready


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipykernel_35/3014747730.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.254200
20,1.908000
30,1.902400
40,1.909400
50,1.789400
60,1.777500
70,1.941900
80,1.472900
90,1.950800
100,1.884500


TrainOutput(global_step=8000, training_loss=1.6759807238578797, metrics={'train_runtime': 1032.9089, 'train_samples_per_second': 30.98, 'train_steps_per_second': 7.745, 'total_flos': 2090336256000000.0, 'train_loss': 1.6759807238578797, 'epoch': 2.0})

In [9]:
model.save_pretrained("./gpt2-finetuned-emotion")
tokenizer.save_pretrained("./gpt2-finetuned-emotion")
print("✅ Model saved")


✅ Model saved


In [10]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./gpt2-finetuned-emotion", tokenizer="./gpt2-finetuned-emotion")

prompt = "<joy> Today I woke up and"
generated = pipe(prompt, max_length=50, num_return_sequences=1)
print("📝 Generated Text:\n", generated[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


📝 Generated Text:
 <joy> Today I woke up and realised that i had missed my precious daughter just last week due to sickbay fever and now i know that her family is too much for me to bear and will leave here next week so i decided to take a


In [11]:
#pushing to hugging face
!pip install --upgrade huggingface_hub
!pip install git-lfs  # for large files


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 8.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 61.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: hf-xet
    Found existing installation: hf-xet 1.1.0
    Uninstalling hf-xet-1.1.0:
      Successfully uninstalled hf-xet-1.1.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.1
    Uninstalling huggingface-hub-0.31.1:
      Successfully uninstalled huggingface-hub-0.31.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
from huggingface_hub import notebook_login

notebook_login()


In [13]:
#creating repo on higging face and pushing
from huggingface_hub import HfApi, create_repo, upload_folder

# Create repo
repo_name = "gpt2mini-finetuned-emotion"  # your choice
create_repo(repo_name, repo_type="model", private=False)  # set private=True if you want to keep it private

# Upload entire folder
upload_folder(
    folder_path="./gpt2-finetuned-emotion",
    path_in_repo=".",
    repo_id="Aditya7492/gpt2mini-finetuned-emotion"
)


Uploading...:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aditya7492/gpt2mini-finetuned-emotion/commit/204a527f2218401c0c1299b1804f18934dc84f48', commit_message='Upload folder using huggingface_hub', commit_description='', oid='204a527f2218401c0c1299b1804f18934dc84f48', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Aditya7492/gpt2mini-finetuned-emotion', endpoint='https://huggingface.co', repo_type='model', repo_id='Aditya7492/gpt2mini-finetuned-emotion'), pr_revision=None, pr_num=None)

In [14]:
#checking model uploaded to hugging face is accessible
from transformers import pipeline

pipe = pipeline("text-generation", model="Aditya7492/gpt2mini-finetuned-emotion")
result = pipe("I'm feeling", max_length=50)
print(result)


config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': "I'm feeling pretty smug about my own personal progress as a user and what an impressive year it has been to date it has been such an amazing year and thanks so much for this opportunity to visit i look forward to seeing you again at some amazing new"}]


In [ ]:
#uploading to github